In [10]:
# import libraries to make get_clip_embeddings work
# import clip
# import torch
import numpy as np
import pandas as pd

 
# warnings.filterwarnings(action = 'ignore')
 
from gensim.models import KeyedVectors


In [15]:
concepts = ['Turtle',
 'Alligator',
 'Lizard',
 'Tortoise',
 'Cobra',
 'Snake',
 'Blindworm',
 'Gecko',
 'Boa Python',
 'Toad',
 'Crocodile',
 'Chameleon',
 'Caiman',
 'Salamander',
 'Dinosaur','Hammer',
 'Screwdriver',
 'Grinding Disc',
 'Vacuum Cleaner',
 'Spanner',
 'Lawn Mower',
 'Axe',
 'Saw',
 'Knife',
 'Nail',
 'Chisel',
 'Shovel',
 'Anvil',
 'Oilcan',
 'Paintbrush']

In [12]:
# helper function to get clip embeddings

import gzip
import html
import os
from functools import lru_cache

import ftfy
import regex as re


@lru_cache()
def bytes_to_unicode():
    """
    Returns list of utf-8 byte and a corresponding list of unicode strings.
    The reversible bpe codes work on unicode strings.
    This means you need a large # of unicode characters in your vocab if you want to avoid UNKs.
    When you're at something like a 10B token dataset you end up needing around 5K for decent coverage.
    This is a signficant percentage of your normal, say, 32K bpe vocab.
    To avoid that, we want lookup tables between utf-8 bytes and unicode strings.
    And avoids mapping to whitespace/control characters the bpe code barfs on.
    """
    bs = list(range(ord("!"), ord("~")+1))+list(range(ord("¡"), ord("¬")+1))+list(range(ord("®"), ord("ÿ")+1))
    cs = bs[:]
    n = 0
    for b in range(2**8):
        if b not in bs:
            bs.append(b)
            cs.append(2**8+n)
            n += 1
    cs = [chr(n) for n in cs]
    return dict(zip(bs, cs))


def get_pairs(word):
    """Return set of symbol pairs in a word.
    Word is represented as tuple of symbols (symbols being variable-length strings).
    """
    pairs = set()
    prev_char = word[0]
    for char in word[1:]:
        pairs.add((prev_char, char))
        prev_char = char
    return pairs


def basic_clean(text):
    text = ftfy.fix_text(text)
    text = html.unescape(html.unescape(text))
    return text.strip()


def whitespace_clean(text):
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text


class SimpleTokenizer(object):
    def __init__(self, bpe_path: str = "bpe_simple_vocab_16e6.txt.gz"):
        self.byte_encoder = bytes_to_unicode()
        self.byte_decoder = {v: k for k, v in self.byte_encoder.items()}
        merges = gzip.open(bpe_path).read().decode("utf-8").split('\n')
        merges = merges[1:49152-256-2+1]
        merges = [tuple(merge.split()) for merge in merges]
        vocab = list(bytes_to_unicode().values())
        vocab = vocab + [v+'' for v in vocab]
        for merge in merges:
            vocab.append(''.join(merge))
        vocab.extend(['<|startoftext|>', '<|endoftext|>'])
        self.encoder = dict(zip(vocab, range(len(vocab))))
        self.decoder = {v: k for k, v in self.encoder.items()}
        self.bpe_ranks = dict(zip(merges, range(len(merges))))
        self.cache = {'<|startoftext|>': '<|startoftext|>', '<|endoftext|>': '<|endoftext|>'}
        self.pat = re.compile(r"""<\|startoftext\|>|<\|endoftext\|>|'s|'t|'re|'ve|'m|'ll|'d|[\p{L}]+|[\p{N}]|[^\s\p{L}\p{N}]+""", re.IGNORECASE)

    def bpe(self, token):
        if token in self.cache:
            return self.cache[token]
        word = tuple(token[:-1]) + ( token[-1] + '',)
        pairs = get_pairs(word)

        if not pairs:
            return token+''

        while True:
            bigram = min(pairs, key = lambda pair: self.bpe_ranks.get(pair, float('inf')))
            if bigram not in self.bpe_ranks:
                break
            first, second = bigram
            new_word = []
            i = 0
            while i < len(word):
                try:
                    j = word.index(first, i)
                    new_word.extend(word[i:j])
                    i = j
                except:
                    new_word.extend(word[i:])
                    break

                if word[i] == first and i < len(word)-1 and word[i+1] == second:
                    new_word.append(first+second)
                    i += 2
                else:
                    new_word.append(word[i])
                    i += 1
            new_word = tuple(new_word)
            word = new_word
            if len(word) == 1:
                break
            else:
                pairs = get_pairs(word)
        word = ' '.join(word)
        self.cache[token] = word
        return word

    def encode(self, text):
        bpe_tokens = []
        text = whitespace_clean(basic_clean(text)).lower()
        for token in re.findall(self.pat, text):
            token = ''.join(self.byte_encoder[b] for b in token.encode('utf-8'))
            bpe_tokens.extend(self.encoder[bpe_token] for bpe_token in self.bpe(token).split(' '))
        return bpe_tokens

    def decode(self, tokens):
        text = ''.join([self.decoder[token] for token in tokens])
        text = bytearray([self.byte_decoder[c] for c in text]).decode('utf-8', errors="replace").replace('', ' ')
        return text


ModuleNotFoundError: No module named 'ftfy'

# get clip text embeddings

In [24]:
# write a function to extract the embeddings from the CLIP model given a list of concepts as text
def get_clip_embeddings(texts):
    # load the model
    model, preprocess = clip.load("ViT-B/32", device="cuda:0")
    tokenizer = SimpleTokenizer()
    text_tokens = [tokenizer.encode("This is " + desc) for desc in texts]
    text_input = torch.zeros(len(text_tokens), model.context_length, dtype=torch.long)
    sot_token = tokenizer.encoder['<|startoftext|>']
    eot_token = tokenizer.encoder['<|endoftext|>']

    for i, tokens in enumerate(text_tokens):
        tokens = [sot_token] + tokens + [eot_token]
        text_input[i, :len(tokens)] = torch.tensor(tokens)

    text_input = text_input.cuda()
    with torch.no_grad():
        text_features = model.encode_text(text_input).float()
    # make a df with the embeddings and the text as row names
    text_features = pd.DataFrame(text_features.cpu().numpy(), index=texts)
    return text_features

In [28]:
clip_embeddings = get_clip_embeddings(concepts)
# save the embeddings
clip_embeddings.to_csv("../../data/clip/clip_embeddings.csv")

# get word2vec embeddings

In [60]:
def load_word2vec_embeddings():
    # load the word2vec model
    model = KeyedVectors.load_word2vec_format('/Users/siddharthsuresh/gensim-data/word2vec-google-news-300/word2vec-google-news-300', binary=True)
    return model

def get_word2vec_embeddings(model, word_list):
    # replace 'Blindworm' with 'worm' as the word2vec model doesn't have the word 'Blindworm'
    word_list = [word.replace('Blindworm', 'worm') for word in word_list]
    word_list = [word.replace('Boa Python', 'python') for word in word_list]
    word_list = [word.replace('Grinding Disc', 'cylindrical_grinding') for word in word_list]
    word_list = [word.replace('Vacuum Cleaner', 'vacuum_cleaner') for word in word_list]
    word_list = [word.replace('Lawn Mower', 'lawn_mower') for word in word_list]
    # make sure all the words are lower case
    word_list = [word.lower() for word in word_list]
    word_list = [word.replace('axe', 'Axe') for word in word_list]
    # get the word2vec representations of the words in the list
    word_vectors = [model[word] for word in word_list]

    # convert the list of word2vec representations to a numpy array
    word_vectors = np.array(word_vectors)
    # make a df with the embeddings and the word_list as row names
    word_vectors = pd.DataFrame(word_vectors, index=word_list)
    return word_vectors

def get_similar_word2vec_embeddings(model, word):
    try:
        return model[word]
    except KeyError:
        closest_word = model.most_similar(word, topn=1)[0][0]
        return model[closest_word]




In [48]:
model = load_word2vec_embeddings()

In [62]:
word2vec_embeddings = get_word2vec_embeddings(model, concepts)
word2vec_embeddings.to_csv("../../data/word2vec/word2vec_embeddings.csv")